In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.mllib.clustering import KMeans
from pyspark.mllib.linalg import Vectors
from operator import add
from pyspark.mllib.clustering import KMeans, KMeansModel
from pyspark.rdd import RDD
import math
import numpy as np

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [5]:
sc = SparkContext.getOrCreate()

rawData = sc.textFile("kddcup.corrected.csv").sample(False, 0.01)
# 1 percent of data , randomly sampled.

In [7]:
# split line by comma and get the last element
last_column_values = rawData.map(lambda x: x.split(',')[-1])

# sort them in descending order
sorted_counts = last_column_values.countByValue().items()
sorted_counts = sorted(sorted_counts, key=lambda x: x[1], reverse=True)

for value, count in sorted_counts:
    print(f"{value}: {count}")

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.hadoop.mapred.InvalidInputException: Input path does not exist: file:/home/aldof/kddcup.corrected.csv
	at org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:287)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:229)
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:315)
	at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:205)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:300)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:296)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:49)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:300)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:296)
	at org.apache.spark.api.python.Pytho

In [19]:
#extract labels and vectors
labelsAndData = rawData.map(lambda line: line.split(',')).map(lambda arr: (arr[-1], Vectors.dense(arr[0:1] + arr[4:-1])))
data = labelsAndData.values().cache()


In [20]:
sample_data = labelsAndData.take(10)  # Take the first 10 elements as a sample
for data_point in sample_data:
    print(data_point)


('normal.', DenseVector([0.0, 224.0, 296.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 7.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 34.0, 125.0, 1.0, 0.0, 0.03, 0.04, 0.0, 0.0, 0.0, 0.0]))
('normal.', DenseVector([0.0, 229.0, 1911.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 13.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 40.0, 131.0, 1.0, 0.0, 0.03, 0.04, 0.0, 0.0, 0.0, 0.0]))
('normal.', DenseVector([0.0, 251.0, 294.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 15.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 15.0, 251.0, 1.0, 0.0, 0.07, 0.03, 0.0, 0.0, 0.0, 0.0]))
('normal.', DenseVector([0.0, 336.0, 520.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 2.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 60.0, 255.0, 1.0, 0.0, 0.02, 0.05, 0.0, 0.0, 0.0, 0.0]))
('normal.', DenseVector([0.0, 266.0, 19790.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 

In [21]:
sample_data = data.take(10)  # Take the first 10 elements as a sample
for data_point in sample_data:
    print(data_point)

[0.0,224.0,296.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,7.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,34.0,125.0,1.0,0.0,0.03,0.04,0.0,0.0,0.0,0.0]
[0.0,229.0,1911.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,13.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,40.0,131.0,1.0,0.0,0.03,0.04,0.0,0.0,0.0,0.0]
[0.0,251.0,294.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,15.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,15.0,251.0,1.0,0.0,0.07,0.03,0.0,0.0,0.0,0.0]
[0.0,336.0,520.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,60.0,255.0,1.0,0.0,0.02,0.05,0.0,0.0,0.0,0.0]
[0.0,266.0,19790.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,7.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,17.0,255.0,1.0,0.0,0.06,0.05,0.0,0.0,0.0,0.0]
[0.0,235.0,3609.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,8.0,0.0,0.0,0.0,0.0,1.0,0.0,0.25,179.0,255.0,1.0,0.0,0.01,0.02,0.0,0.0,0.0,0.0]
[0.0,238.0,4

In [7]:
# pyspark.mllib. is for rdd and pyspark.ml. is for dataframe.

In [22]:
kmeans = KMeans()
model = kmeans.train(data, k = 2)         # k = number of clusters
cluster_centers = model.clusterCenters    # cluster centers

for c in cluster_centers:
    print(c)

[4.70676566e+01 7.83402095e+02 5.73928275e+02 0.00000000e+00
 2.84784378e-04 0.00000000e+00 1.75345810e-02 6.10252238e-05
 1.46257120e-01 1.09845403e-03 0.00000000e+00 0.00000000e+00
 5.71602929e-03 6.91619203e-04 6.10252238e-05 8.13669650e-04
 0.00000000e+00 0.00000000e+00 1.01708706e-03 3.34202563e+02
 2.94571562e+02 1.78253051e-01 1.78375915e-01 5.70616355e-02
 5.71122864e-02 7.89549227e-01 2.11096420e-02 2.72377950e-02
 2.32806530e+02 1.89214382e+02 7.53523596e-01 3.01722945e-02
 6.03213385e-01 6.14727421e-03 1.78470708e-01 1.78260374e-01
 5.71606998e-02 5.69426363e-02]
[5.80000000e+01 8.88000000e+02 1.90086867e+06 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 1.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 1.00000000e+00
 1.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 1.00000000e+00 0.00000000e+00 0.00000

In [23]:
# maps each (label, vector) tuple to (cluster, label)
cluster_label_rdd = labelsAndData.map(lambda x: (model.predict(x[1]), x[0]))

# Count (cluster, label) pair
clusterLabelCount = cluster_label_rdd.countByValue()

for ((cluster, label), count) in sorted(clusterLabelCount.items()):
    print(f"{cluster}{label:18}{count:8}")


0back.                   28
0ipsweep.               103
0neptune.             10791
0nmap.                   23
0normal.               9851
0pod.                     5
0portsweep.              83
0satan.                 144
0smurf.               28115
0spy.                     1
0teardrop.                3
0warezclient.            13
1normal.                  3


#### ------------ Use this to Dump the Non-Normalized Data & Clusters -----------

In [24]:
kmeans = KMeans()
model = kmeans.train(data, k = 100)

# maps each vector to (cluster, vector) and concatenate cluster and vector
sample = data.map(lambda vector: (model.predict(vector), vector.toArray())).map(lambda x: (x[0], ",".join(map(str, x[1]))))
sample.sample(False, 0.01).saveAsTextFile("./kmeans-sample")


###  ------------ Investigate the Average Distance to Closest Centroid  ---------

In [27]:

# calculates the distance between vectors
def distance(a, b):
    return np.sqrt(np.sum((a - b) ** 2))

# calculates the distance to centroid
def distToCentroid(vector, model):
    cluster = model.predict(vector)
    centroid = model.clusterCenters[cluster]
    return distance(centroid, vector)

# calculates clustering score
def clusteringScore(data, k):
    kmeans = KMeans.train(data, k)
    return data.map(lambda vector: distToCentroid(vector, kmeans)).mean()

results = []
for k in range(30, 61, 10):
    score = clusteringScore(data, k)
    results.append((k, score))

for r in results:
    print(r)


(30, 269.62675538366244)
(40, 133.9497731323642)
(50, 186.1093867782195)
(60, 232.90671692144602)


### ------------ Normalize the Data and Compare Clustering Scores --------------

In [28]:
def normalize(data):
    data_as_array = data.map(lambda x: x.toArray())
    num_cols = len(data_as_array.first())
    n = data_as_array.count()
    sums = data_as_array.reduce(lambda a, b: [x + y for x, y in zip(a, b)])
    sum_squares = data_as_array.aggregate([0.0] * num_cols,
                                          lambda a, b: [x + y * y for x, y in zip(a, b)],
                                          lambda a, b: [x + y for x, y in zip(a, b)])
    stdevs = [math.sqrt(n * sum_sq - sum_ * sum_) / n for sum_sq, sum_ in zip(sum_squares, sums)]
    means = [sum_ / n for sum_ in sums]

    def normalize_vector(vector):
        normalized_array = [(value - mean) / stdev if stdev > 0 else value - mean
                            for value, mean, stdev in zip(vector.toArray(), means, stdevs)]
        return Vectors.dense(normalized_array)

    return normalize_vector

normalized_data = data.map(normalize(data)).cache()


clustering_scores = [(k, clusteringScore(normalized_data, k)) for k in range(60, 121, 10)]


for score in clustering_scores:
    print(score)

normalized_data.unpersist()



(60, 0.36583083690398577)
(70, 0.3306916741201658)
(80, 0.30956256007676186)
(90, 0.2907341227802317)
(100, 0.2770668724341242)
(110, 0.24904216657098613)
(120, 0.23244947833475677)


PythonRDD[625] at RDD at PythonRDD.scala:53

### ------------ Switch to One-Hot Encoding of Categorical Attributes  ---------

In [29]:

def onehot(raw_data):
    splitData = raw_data.map(lambda x: x.split(','))
    protocols = splitData.map(lambda x: x[1]).distinct().zipWithIndex().collectAsMap()
    print("PROTOCOLS", protocols)
    services = splitData.map(lambda x: x[2]).distinct().zipWithIndex().collectAsMap()
    print("SERVICES", services)
    tcpStates = splitData.map(lambda x: x[3]).distinct().zipWithIndex().collectAsMap()
    print("TCPSTATES", tcpStates)

    def encode_line(line):
        buffer = line.split(',')
        protocol = buffer.pop(1)
        service = buffer.pop(1)
        tcpState = buffer.pop(1)
        label = buffer.pop(-1)
        vector = [float(x) for x in buffer]

        newProtocolFeatures = [0.0] * len(protocols)
        newProtocolFeatures[protocols[protocol]] = 1.0
        newServiceFeatures = [0.0] * len(services)
        newServiceFeatures[services[service]] = 1.0
        newTcpStateFeatures = [0.0] * len(tcpStates)
        newTcpStateFeatures[tcpStates[tcpState]] = 1.0

        vector[1:1] = newTcpStateFeatures
        vector[1:1] = newServiceFeatures
        vector[1:1] = newProtocolFeatures

        return label, Vectors.dense(vector)

    return encode_line

print("2")



2


### ------------- Dump the Normalized and One-Hot Encoded Data  ----------------

In [30]:

parseFunction = onehot(rawData)
labelsAndData = rawData.map(parseFunction)
normalizedLabelsAndData = labelsAndData.mapValues(normalize(labelsAndData.values())).cache()

kmeans = KMeans()
model = kmeans.train(normalizedLabelsAndData.values(), k = 100)
sample = normalizedLabelsAndData.values().map(lambda vector: str(model.predict(vector)) + "," + ",".join(map(str, vector.toArray()))).sample(False, 0.01)
sample.saveAsTextFile("./kmeans-sample-normalized")
normalizedLabelsAndData.unpersist()


PROTOCOLS {'tcp': 0, 'udp': 1, 'icmp': 2}


SERVICES {'time': 0, 'courier': 1, 'pop_3': 2, 'smtp': 3, 'ftp': 4, 'http_443': 5, 'login': 6, 'bgp': 7, 'IRC': 8, 'link': 9, 'shell': 10, 'csnet_ns': 11, 'gopher': 12, 'Z39_50': 13, 'eco_i': 14, 'efs': 15, 'rje': 16, 'name': 17, 'systat': 18, 'urh_i': 19, 'uucp': 20, 'domain': 21, 'sunrpc': 22, 'whois': 23, 'remote_job': 24, 'discard': 25, 'finger': 26, 'ntp_u': 27, 'other': 28, 'ldap': 29, 'kshell': 30, 'echo': 31, 'ctf': 32, 'sql_net': 33, 'iso_tsap': 34, 'nntp': 35, 'ssh': 36, 'hostnames': 37, 'telnet': 38, 'klogin': 39, 'mtp': 40, 'netbios_dgm': 41, 'urp_i': 42, 'supdup': 43, 'nnsp': 44, 'netbios_ssn': 45, 'imap4': 46, 'netstat': 47, 'domain_u': 48, 'auth': 49, 'private': 50, 'netbios_ns': 51, 'exec': 52, 'uucp_path': 53, 'http': 54, 'ecr_i': 55, 'ftp_data': 56, 'vmnet': 57, 'pop_2': 58, 'printer': 59, 'daytime': 60}


TCPSTATES {'OTH': 0, 'S3': 1, 'SF': 2, 'REJ': 3, 'RSTO': 4, 'SH': 5, 'S0': 6, 'S2': 7, 'S1': 8, 'RSTR': 9}


PythonRDD[1227] at RDD at PythonRDD.scala:53

### ------------ Switch to Entropy-Based Clustering Scores  --------------------

In [31]:

def entropy(counts):
    values = [c for c in counts if c > 0]
    n = sum(values)
    return sum(-p * math.log(p) for v in values for p in [v / n])

def clusteringScoreByEntropy(normalizedData, k):
    normalizedData.cache()

    kmeans = KMeans()
    model = kmeans.train(normalizedData.values(), k)

    normalizedData.unpersist()

    labelsAndClusters = normalizedData.mapValues(lambda x: model.predict(x))
    clustersAndLabels = labelsAndClusters.map(lambda x: (x[1], x[0]))
    labelsInCluster = clustersAndLabels.groupByKey().values()
    labelCounts = labelsInCluster.map(lambda x: list(map(len, x))).map(lambda x: [sum(x)] + x)
    n = normalizedData.count()

    return labelCounts.map(lambda m: sum(m) * entropy(m)).sum() / n


parseFunction = onehot(rawData)
labelsAndData = rawData.map(parseFunction)
normalizeFunction = normalize(labelsAndData.values())
normalizedLabelsAndData = labelsAndData.mapValues(normalizeFunction).cache()

results = [(k, clusteringScoreByEntropy(normalizedLabelsAndData, k)) for k in range(120, 161, 10)]
for r in results:
    print(r)


PROTOCOLS {'tcp': 0, 'udp': 1, 'icmp': 2}


SERVICES {'time': 0, 'courier': 1, 'pop_3': 2, 'smtp': 3, 'ftp': 4, 'http_443': 5, 'login': 6, 'bgp': 7, 'IRC': 8, 'link': 9, 'shell': 10, 'csnet_ns': 11, 'gopher': 12, 'Z39_50': 13, 'eco_i': 14, 'efs': 15, 'rje': 16, 'name': 17, 'systat': 18, 'urh_i': 19, 'uucp': 20, 'domain': 21, 'sunrpc': 22, 'whois': 23, 'remote_job': 24, 'discard': 25, 'finger': 26, 'ntp_u': 27, 'other': 28, 'ldap': 29, 'kshell': 30, 'echo': 31, 'ctf': 32, 'sql_net': 33, 'iso_tsap': 34, 'nntp': 35, 'ssh': 36, 'hostnames': 37, 'telnet': 38, 'klogin': 39, 'mtp': 40, 'netbios_dgm': 41, 'urp_i': 42, 'supdup': 43, 'nnsp': 44, 'netbios_ssn': 45, 'imap4': 46, 'netstat': 47, 'domain_u': 48, 'auth': 49, 'private': 50, 'netbios_ns': 51, 'exec': 52, 'uucp_path': 53, 'http': 54, 'ecr_i': 55, 'ftp_data': 56, 'vmnet': 57, 'pop_2': 58, 'printer': 59, 'daytime': 60}


TCPSTATES {'OTH': 0, 'S3': 1, 'SF': 2, 'REJ': 3, 'RSTO': 4, 'SH': 5, 'S0': 6, 'S2': 7, 'S1': 8, 'RSTR': 9}


(120, 68.66310225148231)
(130, 67.51713942968327)
(140, 68.51568334084513)
(150, 67.4680657891172)
(160, 66.65860496116079)


In [32]:

model = KMeans.train(normalizedLabelsAndData.values(), 150)


clusterLabelCount = labelsAndData.map(lambda x: (model.predict(normalizeFunction(x[1])), x[0])) \
    .countByValue()


for ((cluster, label), count) in sorted(clusterLabelCount.items()):
    print(f"{cluster:<1}{label:18}{count:8}")

normalizedLabelsAndData.unpersist()

0nmap.                    2
0normal.                728
0satan.                  11
1normal.               2315
2neptune.              8230
3ipsweep.                24
4neptune.                 5
5normal.                  1
6portsweep.              17
6satan.                 106
7neptune.                 2
8normal.                  2
9normal.                 31
9smurf.                1286
10neptune.                16
11neptune.                11
12neptune.                 8
13normal.               1801
14neptune.                 6
15neptune.                 7
16neptune.                11
17neptune.                15
18normal.                  1
19ipsweep.                 2
19neptune.              1021
19portsweep.               3
20neptune.                11
21neptune.                12
22normal.                  1
22portsweep.               2
23neptune.                14
23normal.                  1
24neptune.               928
24portsweep.               2
25normal.                227

PythonRDD[1383] at RDD at PythonRDD.scala:53

In [33]:
def anomaly(data, normalizeFunction):
    normalizedData = data.map(normalizeFunction)
    normalizedData.cache()

    kmeans = KMeans()
    model = kmeans.train(normalizedData, k=150)

    normalizedData.unpersist()

    distances = normalizedData.map(lambda vector: distToCentroid(vector, model))
    threshold = distances.top(100)[-1]

    return lambda vector: distToCentroid(normalizeFunction(vector), model) > threshold

parseFunction = onehot(rawData)
originalAndData = rawData.map(lambda line: (line, parseFunction(line)[1]))
data = originalAndData.values()
normalizeFunction = normalize(data)
anomalyFunction = anomaly(data, normalizeFunction)
anomalies = originalAndData.filter(lambda x: anomalyFunction(x[1])).keys()
anomalies.take(10)


PROTOCOLS {'tcp': 0, 'udp': 1, 'icmp': 2}


SERVICES {'time': 0, 'courier': 1, 'pop_3': 2, 'smtp': 3, 'ftp': 4, 'http_443': 5, 'login': 6, 'bgp': 7, 'IRC': 8, 'link': 9, 'shell': 10, 'csnet_ns': 11, 'gopher': 12, 'Z39_50': 13, 'eco_i': 14, 'efs': 15, 'rje': 16, 'name': 17, 'systat': 18, 'urh_i': 19, 'uucp': 20, 'domain': 21, 'sunrpc': 22, 'whois': 23, 'remote_job': 24, 'discard': 25, 'finger': 26, 'ntp_u': 27, 'other': 28, 'ldap': 29, 'kshell': 30, 'echo': 31, 'ctf': 32, 'sql_net': 33, 'iso_tsap': 34, 'nntp': 35, 'ssh': 36, 'hostnames': 37, 'telnet': 38, 'klogin': 39, 'mtp': 40, 'netbios_dgm': 41, 'urp_i': 42, 'supdup': 43, 'nnsp': 44, 'netbios_ssn': 45, 'imap4': 46, 'netstat': 47, 'domain_u': 48, 'auth': 49, 'private': 50, 'netbios_ns': 51, 'exec': 52, 'uucp_path': 53, 'http': 54, 'ecr_i': 55, 'ftp_data': 56, 'vmnet': 57, 'pop_2': 58, 'printer': 59, 'daytime': 60}


TCPSTATES {'OTH': 0, 'S3': 1, 'SF': 2, 'REJ': 3, 'RSTO': 4, 'SH': 5, 'S0': 6, 'S2': 7, 'S1': 8, 'RSTR': 9}


['0,tcp,auth,SF,9,37,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,1,0.00,0.00,0.00,0.00,0.50,1.00,0.00,69,22,0.07,0.06,0.01,0.18,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,other,REJ,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.00,0.00,1.00,1.00,1.00,0.00,0.00,2,2,1.00,0.00,0.50,0.00,0.00,0.00,1.00,1.00,normal.',
 '0,tcp,other,REJ,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.00,0.00,1.00,1.00,1.00,0.00,0.00,3,3,1.00,0.00,0.33,0.00,0.00,0.00,1.00,1.00,normal.',
 '27,tcp,ftp,SF,926,2720,0,0,0,19,0,1,0,0,0,0,0,0,0,0,0,1,1,1,0.00,0.00,0.00,0.00,1.00,0.00,0.00,143,44,0.31,0.02,0.01,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,auth,SF,9,37,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,1,0.00,0.00,0.00,0.00,0.50,1.00,0.00,234,16,0.06,0.02,0.00,0.12,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,auth,SF,10,39,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,1,0.00,0.00,0.00,0.00,0.50,1.00,0.00,104,5,0.05,0.05,0.01,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,other,REJ,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.00,0.00,1.00,1.00,1.00,0.00,0.00,8,8,1.00,0.00

In [34]:
fractionofanomalies = anomalies.count() / rawData.count() * 100
print("Fraction of anomalies: {:.2f}%".format(fractionofanomalies))

Fraction of anomalies: 0.20%
